In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import math

K_number = "5"
model1 = tf.keras.models.load_model(r"F:\Courses\Paper\Mine\Proposing an empirical motion-time pattern for human gaze behavior in different social situations using Deep Neural Networks\Results\Models\LSTMBody-K1-K_max10-sequence24-delta1.h5")

mean = np.array([  0.78103808,   1.18988002,   0.33757173,   0.27666927,   0.33020344,
                 -47.10974178,   0.06135889,   0.76323683,   1.13328117,   0.34031038,
                   0.41373239,   0.33176839, -24.64593114,   0.16901408,   0.77992958,
                   1.14358372,   0.32961659,   0.34963485,   0.31781429,  25.04499218,
                   0.1691445,    0.77562598,   1.18792384,   0.35609025,   0.30549035,
                   0.34709181,  46.83000782,   0.07100939])

STD = np.array([ 0.41354274,  0.76925252,  0.47288165,  0.44735153,  0.47028622, 25.01631484,
                  0.41562948,  0.42509572,  0.76749179,  0.47381349,  0.49250168,  0.4708483,
                 15.79101793,  0.66666253,  0.4142939,   0.75040541,  0.47007392,  0.47685461,
                  0.46562686, 15.29470315,  0.67247272,  0.41716941,  0.7754718,   0.47884234,
                  0.4606148,   0.47604526, 25.27220849,  0.44034229])




#Right Hand   #Hand  X=Positions[4] Y=Positions[5] Z=Positions[6]   #Elbow X=Positions[7] Y=Positions[8] Z=Positions[9] #Shoulder X=Positions[10] Y=Positions[11] Z=Positions[12]
#Head X=Positions[1] Y=Positions[2] Z=Positions[3]
#Left Hand   #Hand  X=Positions[13] Y=Positions[14] Z=Positions[15]   #Elbow X=Positions[16] Y=Positions[17] Z=Positions[18] #Shoulder X=Positions[19] Y=Positions[20] Z=Positions[21]

def pointing(Positions):
    if abs(Positions[5]-Positions[11])<0.1 or abs(Positions[14]-Positions[20])<0.1:
        if Positions[5]-Positions[8]<0.1 or Positions[14]-Positions[17]<0.1:
            return 1
    return 0



def waving(Positions):
    if Positions[5]-Positions[8]>0.2 or Positions[14]-Positions[17]>0.2:
        return 1
    return 0


def talking(Audio):
    for i in range(24):
        if Audio[i,0,1]<-20:
            return 4
        elif Audio[i,0,1]>-20 and Audio[i,0,1]<0:
            return 3
        elif Audio[i,0,1]<20 and Audio[i,0,1]>0:
            return 2
        elif Audio[i,0,1]>20:
            return 1
    return 0

def Enter(Positions):
    unique_positions = np.unique(Positions)
    std = np.std(unique_positions)
    if std > 1:
        # Mask the zeros
        A = abs(Positions)
        masked_arr = np.ma.masked_equal(A, 0)
        min_val = masked_arr.min()
        min_index = masked_arr.argmin()
        max_val = masked_arr.max()
        max_index = masked_arr.argmax()
        if max_index - min_index<0:
            return 1
        else:
            return 3
    return 0



def TT(Positions):
    Positions_new = np.zeros((Positions.shape[0], Positions.shape[1], Positions.shape[2]))
    Persons = np.zeros(4)
    for i in range(4):
        Persons[i] = Positions[-1,i,0]
        
    for j in range(24):
        for i in range(4):
            if np.array_equal(Persons[i], Positions[j,i,0]):
                Positions_new[j,i] = np.array(Positions[j,i])
            else:
                diff = np.setdiff1d(Positions[j,:,0], Persons)
                #diff_indices = np.where(np.isin(Positions[j,:,0], diff))
                if diff.shape[0]>0:
                    idx = np.where(Persons == 0)[0][0]
                    Persons[idx] = diff
                    
                if Persons[i] in Positions[j,:,0]:
                    if Persons[i]==0:
                        continue
                    index = np.where(Positions[j,:,0] == Persons[i])[0][0]
                    Positions_new[j,i] = np.array(Positions[j,index])
                
    return Positions_new

def RR(Positions):
    check = np.zeros(4)
    Positions_new = np.zeros((Positions.shape[0], Positions.shape[1], Positions.shape[2]))

    if np.array_equal(check, np.zeros(4)):
        for i in range(24):
            for j in range(4):
                if Positions[i,j,1] > 0.5: #Person Number 1
                    Positions_new[i,0] = Positions[i,j]
                elif Positions[i,j,1] < 0.5 and Positions[i,j,1] > 0: #Person Number 1
                    Positions_new[i,1] = np.array(Positions[i,j])
                    
                elif Positions[i,j,1] < 0 and Positions[i,j,1] > -0.5: #Person Number 3
                    Positions_new[i,2] = Positions[i,j]        
                    
                elif Positions[i,j,1] < -0.5: #Person Number 3
                    Positions_new[i,3] = Positions[i,j]   
    return Positions_new

def Create_Input(Positions, Audio):
    Input = np.zeros((1,24,28))
    for i in range(24):
        for j in range(4):
            if Positions[i,j,0]>0 and Positions[i,j,3]<3: #Persons who are in the screen
            #if Positions[i,j,0]>0: #Person Number one
                Input[0,i,7*j] = 1
                if Positions[i,j,3]>2.1: #Far or Near
                    Input[0,i,7*j+1] = 2
                else:
                    Input[0,i,7*j+1] = 1

                if waving(Positions[i,j]) == 1:
                    Input[0,i,7*j+2] = 1
                if pointing(Positions[i,j]) == 1:
                    Input[0,i,7*j+3] = 1

                if Enter(Positions[:,j,1]) > 0:
                    Input[0,i,7*j+6] = Enter(Positions[:,j,1])

                if talking(Audio) == j+1:
                    Input[0,i,7*j+4] = 1

                

                # Calculate the arctangent of a value
                
                atan_value = -2*math.atan(Positions[i,j,1]/Positions[i,j,3])
                Input[0,i,7*j+5] = math.degrees(atan_value)
                        
    return Input

def Normalize(Input, Mean, STD):
    Input_normalized = np.zeros((Input.shape[0], Input.shape[1], Input.shape[2]))
    for i in range(Input.shape[1]):
        Input_normalized[0,i] = (np.array(Input[0,i])-np.array(mean))/np.array(STD)
    return Input_normalized

def Predict(y_pred, Input):
    Persons = np.zeros(4)
    for i in range(4):
        Persons[i] = np.array(Input[0,-1,7*i])
    indices = np.where(Persons == 1)[0]
    y_pred[0,4] = 0
    for i in range(4):
        max_index = np.argmax(y_pred)
        if max_index in indices:
            return max_index + 1
        else:
            y_pred[0,max_index] = 0
    return 0

input = np.zeros((1,24,28))
Positions = np.zeros((24,4,22), dtype="float")
Audio = np.zeros((24,1,2), dtype="float")

for i in range(24):
    with open('F:\Courses\Paper\Mine\Proposing an empirical motion-time pattern for human gaze behavior in different social situations using Deep Neural Networks\Results/Pos.txt') as file:
        B = file.read()
        while len(B)==0:
            B = file.read()
        array_1d = np.array([float(x.strip()) for x in B.split(',')])
        # Reshape the 1D array to a 4x22 array
        array_2d = array_1d.reshape(4, 22)
        Positions[i] = array_2d
    with open('F:\Courses\Paper\Mine\Proposing an empirical motion-time pattern for human gaze behavior in different social situations using Deep Neural Networks\Results/Audio.txt') as file:
        B = file.read()
        while len(B)==0:
            B = file.read()
        array_1d = np.array([float(x.strip()) for x in B.split(',')])
        # Reshape the 1D array to a 1x2 array
        array_2d = array_1d.reshape(1, 2)
        Audio[i] = array_2d
        



                    


In [2]:
while True:
    Positions[0:23,:,:] = np.array(Positions[1:24,:,:])
    Audio[0:23,:,:] = np.array(Audio[1:24,:,:])
    with open('F:\Courses\Paper\Mine\Proposing an empirical motion-time pattern for human gaze behavior in different social situations using Deep Neural Networks\Results/Pos.txt') as file:
        B = file.read()
        while len(B)==0:
            B = file.read()
        array_1d = np.array([float(x.strip()) for x in B.split(',')])
        # Reshape the 1D array to a 4x22 array
        array_2d = array_1d.reshape(4, 22)
        Positions[-1] = array_2d
            
    with open('F:\Courses\Paper\Mine\Proposing an empirical motion-time pattern for human gaze behavior in different social situations using Deep Neural Networks\Results/Audio.txt') as file:
        B = file.read()
        while len(B)==0:
            B = file.read()
        array_1d = np.array([float(x.strip()) for x in B.split(',')])
        # Reshape the 1D array to a 1x2 array
        array_2d = array_1d.reshape(1, 2)
        Audio[-1] = array_2d
        
    Positions = TT(Positions)
    positions2 = RR(Positions)
    Input = Create_Input(positions2, Audio)
    Input_normalized = Normalize(Input, mean, STD) 
    y_pred = model1.predict(Input_normalized, verbose=0)
    Person_look = Predict(y_pred, Input)
    if Person_look>0:
        print(Input[0,-1,3+(Person_look-1)*7])
        print(Input[0,-1,5+(Person_look-1)*7]*-1, flush=True)

            

0.0
29.580426467949252
0.0
23.916929451729633
0.0
16.167581787754795
0.0
12.718041759793335
0.0
10.43037666023838
0.0
9.509823217469126
0.0
8.94500199610897
0.0
8.27947361058162
0.0
8.124661079609986
0.0
8.125487240513246
0.0
8.220673093858355
0.0
8.523176250078729
0.0
8.982502372510067
0.0
11.66340716879031
0.0
14.128426756678268
0.0
17.013288508188342
0.0
33.52574812380488
0.0
45.92981230716503
0.0
52.29448190927035
0.0
56.69678547123272
0.0
58.191109922028986
0.0
58.37953116200203
0.0
58.64473163193236
0.0
58.9894503657494
0.0
59.019259591661886
0.0
58.910378115024194
0.0
58.25995074371422
0.0
57.74029860174321
0.0
58.80356574859255
0.0
62.292460003384875
0.0
62.34689684116874
0.0
58.330944631108714
0.0
49.324042434410984
0.0
62.83610860358399
0.0
60.71477413153128
0.0
60.578048120700764
0.0
59.67898015667037
0.0
59.333052617139444
0.0
59.19517464087735
0.0
60.22418631863123
0.0
58.85492730967396
0.0
30.362050666216433
0.0
20.517588365493378
0.0
18.547729218731043
0.0
17.10396690387


KeyboardInterrupt



In [ ]:
Positions.shape

In [ ]:
Audio.shape

In [ ]:
Positions[0,0,1]<0.4

In [ ]:
positions2

In [ ]:
y_pred

In [ ]:
Input[-1]